<a href="https://colab.research.google.com/github/kevinkamm/RatingML/blob/main/DeepNeuralNetwork/RatingNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/kevinkamm/RatingML.git
!git remote add origin https://github.com/kevinkamm/RatingML.git

Cloning into 'RatingML'...
remote: Enumerating objects: 458, done.
remote: Counting objects: 100% (458/458), done.
remote: Compressing objects: 100% (336/336), done.
remote: Total 458 (delta 149), reused 417 (delta 118), pack-reused 0
Receiving objects: 100% (458/458), 1.85 MiB | 6.43 MiB/s, done.
Resolving deltas: 100% (149/149), done.
fatal: not a git repository (or any of the parent directories): .git


In [ ]:
ls

RatingML/  sample_data/


In [ ]:
cd 'RatingML'

/content/RatingML


In [ ]:
import tensorflow
print(tensorflow.__version__)

2.8.0


In [ ]:
# from RatingTimeGAN import timeGAN as tg
# from RatingTimeGAN import loadRatingMatrices as lrm
# from RatingTimeGAN import brownianMotion as bm
from RatingTimeGAN import TimeGAN,BrownianMotion,getTimeIndex,RML

import numpy as np
import tensorflow as tf

from typing import List

import time as timer

#tf.config.set_visible_devices([], 'GPU')
print(tf.config.experimental.get_synchronous_execution())
print(tf.config.experimental.list_physical_devices())
print(tf.config.threading.get_inter_op_parallelism_threads())
print(tf.config.threading.get_intra_op_parallelism_threads())

'Data type for computations'
# use single precision for GeForce GPUs
dtype = np.float32

# seed for reproducibility
seed = 0
tf.random.set_seed(seed)

'Parameters for Brownian motion'
# time steps of Brownian motion, has to be such that mod(N-1,12)=0
N = 5 * 12 + 1
# trajectories of Brownian motion will be equal to batch_size for training
# M = batch_size = 1

'Load rating matrices'
# choose between 1,3,6,12 months
times = np.array([1, 3, 6, 12])
lenSeq = times.size
T = times[-1] / 12

# Brownian motion class with fixed datatype
BM = BrownianMotion(T, N, dtype=dtype, seed=seed)
timeIndices = getTimeIndex(T, N, times / 12)

# relative path to rating matrices:
filePaths: List[str] = ['Data/'+'SP_' + str(x) + '_month_small' for x in times]
# exclude default row, don't change
# excludeDefaultRow = False
# permuteTimeSeries, don't change
# permuteTimeSeries = True
# load rating matrices
RML = RML(filePaths,
              dtype=dtype)
print('Load data')
ticRML = timer.time()
RML.loadData()
ctimeRML = timer.time() - ticRML
print(f'Elapsed time for loading data {ctimeRML} s.')

'Build GAN'
# training data
rm_train = RML.tfData()
print(f'Data shape: (Data,Time Seq,From Rating*To Rating)={rm_train.shape}')
# number of ratings
Krows = RML.Krows
Kcols = RML.Kcols
# batch size
batch_size = 128

# buffer size should be greater or equal number of data,
# is only important if data doesn't fit in RAM
buffer_size = rm_train.shape[0]

dataset = tf.data.Dataset.from_tensor_slices(rm_train)
dataset = dataset.shuffle(buffer_size, reshuffle_each_iteration=True).batch(batch_size)

epochs = 20
saveDir = 'RatingTimeGAN/modelParams/Colab'
tGAN = TimeGAN(lenSeq, Krows, Kcols, batch_size, BM, timeIndices, dtype=dtype)
tGAN.trainTimeGAN(dataset, epochs, loadDir=saveDir)
tGAN.save(saveDir)
samples = tGAN.sample(10)
print(samples.shape)
for wi in range(0, 3):
    print(f'Trajectory {wi}\n')
    for ti in range(0, samples.shape[1]):
        print(f'Time {timeIndices[ti]}')
        print(samples[wi, ti, :, :])
        print(np.sum(samples[wi, ti, :, :], axis=1))

saveCSVDir = 'RatingTimeGAN/CSV'
print('Save CSV')
ticCSV=timer.time()
tGAN.exportToCSV(2,saveCSVDir,ratings = RML.ratings)
ctimeCSV=timer.time()-ticCSV
print(f'Elapsed time for saving CSV files: {ctimeCSV} s')

True
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
0
0
Load data
Elapsed time for loading data 4.283725261688232 s.
Data shape: (Data,Time Seq,From Rating*To Rating)=(629856, 4, 16)


INFO:tensorflow:Assets written to: /content/RatingML/RatingTimeGAN/modelParams/Colab/AE560_G854_lenSeq4_batch128_epochs20/embeddedGenerator/assets


INFO:tensorflow:Assets written to: /content/RatingML/RatingTimeGAN/modelParams/Colab/AE560_G854_lenSeq4_batch128_epochs20/embeddedGenerator/assets
Synthetic data generation: 100%|██████████| 10/10 [00:04<00:00,  2.39it/s]


(1280, 4, 4, 4)
Trajectory 0

Time 5
[[9.9414045e-01 5.8828811e-03 6.6160565e-06 3.0921899e-07]
 [1.0395615e-03 9.9658006e-01 2.4588432e-03 3.0710893e-05]
 [1.6595744e-05 4.1161403e-03 9.8655409e-01 9.0501653e-03]
 [2.4027128e-07 1.9028822e-07 1.4560077e-07 9.9999988e-01]]
[1.0000303 1.0001092 0.999737  1.0000005]
Time 15
[[9.9028343e-01 9.7226072e-03 1.8113400e-05 9.1955957e-07]
 [1.6898537e-03 9.9434328e-01 3.9570983e-03 7.3341304e-05]
 [3.9740480e-05 7.2807181e-03 9.7559047e-01 1.6250012e-02]
 [5.4830929e-07 4.5745728e-07 3.7822292e-07 9.9999964e-01]]
[1.0000252  1.0000635  0.99916095 1.0000011 ]
Time 30
[[9.7498453e-01 2.4908675e-02 1.0274399e-04 4.9634441e-06]
 [4.4010133e-03 9.8642391e-01 8.9946818e-03 2.4095489e-04]
 [1.8826734e-04 2.2452824e-02 9.4090033e-01 3.5172977e-02]
 [2.0426448e-06 1.8196464e-06 1.7536124e-06 9.9999833e-01]]
[1.000001  1.0000606 0.9987144 1.0000039]
Time 60
[[9.4309676e-01 5.6361575e-02 5.4543931e-04 2.9719784e-05]
 [1.0000689e-02 9.6912986e-01 1.9699190

Synthetic data generation: 100%|██████████| 2/2 [00:00<00:00, 15.59it/s]


Elapsed time for saving CSV files: 1.8265504837036133 s


In [ ]:
!zip -r /content/modelParams.zip /content/RatingML/RatingTimeGAN/modelParams/

In [ ]:
!git config --global user.email 'kevinkamm@t-online.de'
!git config --global user.name 'Kevin'
!git add -A
!git commit -m 'new model parameters'
!git pull
!git push 

[main 424d7bc] new model parameters
 6 files changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 RatingTimeGAN/CSV/AE560_G854_lenSeq4_batch128_epochs20_N61.zip
 rewrite RatingTimeGAN/__pycache__/brownianMotion.cpython-37.pyc (72%)
 rewrite RatingTimeGAN/__pycache__/timeGAN.cpython-37.pyc (74%)
Already up to date.
Counting objects: 15, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (15/15), done.
Writing objects: 100% (15/15), 397.41 KiB | 7.95 MiB/s, done.
Total 15 (delta 6), reused 0 (delta 0)
remote: Resolving deltas: 100% (6/6), completed with 6 local objects.
To https://github.com/kevinkamm/RatingML.git
   742685f..424d7bc  main -> main
